In [ ]:
%pip install -r requirements.txt

In [35]:
import dagshub
import datetime
import mlflow
import torch
import time
import os
import shutil
import random
import yaml
import pandas as pd
from pathlib import Path
from roboflow import Roboflow
from ultralytics import YOLO, settings

In [ ]:
rf = Roboflow(api_key="aRTRw0ORiLGJdj7RF2gR")
project = rf.workspace("abiya-thesis").project("tennis-player-and-tennis-ball-ei0vm")
version = project.version(6)
dataset = version.download("yolov12", location="datasets/object-detection")

In [2]:
rf = Roboflow(api_key="aRTRw0ORiLGJdj7RF2gR")
project = rf.workspace("abiya-thesis").project("tennis-player-and-tennis-ball-ei0vm")
version = project.version(7)
dataset = version.download("yolov12",location="datasets/object-detection-tile")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to datasets/object-detection-tile in yolov12:: 100%|██████████| 21068/21068 [00:07<00:00, 2822.45it/s]


In [3]:
dagshub.init(repo_owner='abiyamf', repo_name='courtPlay', mlflow=True)
settings.update({"mlflow": True})

Accessing as nugrohorahmanto24

Initialized MLflow to track repo "abiyamf/courtPlay"

Repository abiyamf/courtPlay initialized!

In [59]:
# ==========================
# 🔧 Global Config
# ==========================
os.environ["MLFLOW_KEEP_RUN_ACTIVE"] = "True"

time_now = datetime.datetime.now().strftime("%m-%d_%H-%M")
gpu_name = torch.cuda.get_device_name(0)

# Project Info
projectType = "object-detection"
datasetVersion = "6"
yoloVersion = "yolo11"
modelSize = "n"

# Training Params
epochs = 2
batch = -1
imgsz = 640
data_path = "datasets/object-detection/data.yaml"
project_path = f"results/object-detection/{modelSize}"

# Toggle custom augmentation
use_custom_aug = True

In [60]:
def format_duration(seconds: float) -> str:
    """Convert seconds into 'Hh Mm Ss' format."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours}h {minutes}m {secs}s"

def save_class_metrics(val_results):
    # Ambil nama class
    names = val_results.names

    # Data per-class
    data = {
        "class": [names[i] for i in range(len(names))],
        "precision": val_results.box.p.tolist(),
        "recall": val_results.box.r.tolist(),
        "mAP50": val_results.box.ap50.tolist(),
        "mAP50-95": val_results.box.maps.tolist(),
    }

    # Tambahkan baris agregat "all"
    all_row = {
        "class": "all",
        "precision": float(val_results.box.mp),
        "recall": float(val_results.box.mr),
        "mAP50": float(val_results.box.map50),
        "mAP50-95": float(val_results.box.map),
    }

    for key in data.keys():
        data[key].insert(0, all_row[key])

    # Buat DataFrame dan simpan
    df = pd.DataFrame(data)
    csv_path = os.path.join(str(val_results.save_dir), "class_metrics.csv")
    df.to_csv(csv_path, index=False)

    return csv_path

In [61]:
# ==========================
# 🚀 Training Routine
# ==========================
model = YOLO(f"{yoloVersion}{modelSize}.pt")
mlflow.set_experiment(f"courtPlay-{modelSize}")

start_time = time.time()

# Set augmentations (default vs custom)
augmentations = {}
if use_custom_aug:
    augmentations = {
        "mosaic": 0.4,
        "scale": 0.25,
        "translate": 0.05,
        "hsv_s": 0.4,
        "hsv_v": 0.2,
        "mixup": 0.1,
        "flipud": 0.0,
        "cutmix": 0.0,
        "shear": 0.0,
        "perspective": 0.0,
    }

with mlflow.start_run(run_name=f"{time_now}"):

    # Log global parameters
    mlflow.log_params({
        "project type": projectType,
        "dataset version": datasetVersion,
        "yolo version": yoloVersion,
        "model size": modelSize,
        "gpu": gpu_name,
        "epochs": epochs,
        "batch": batch,
        "imgsz": imgsz,
        "use_custom_aug": use_custom_aug,
    })

    # Train
    results = model.train(
        data=data_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        name=f"{time_now}",
        project=project_path,
        exist_ok=True,
        **augmentations
    )

    # Log training time
    mlflow.log_params({
        "training time": format_duration(time.time() - start_time),
    })

    # Run validation + log class metrics
    mlflow.log_artifact(save_class_metrics(results), artifact_path="class_metrics")

mlflow.end_run()

New https://pypi.org/project/ultralytics/8.3.204 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.179  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/object-detection/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.2, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=0.4, multi_scale=False, name=10-05_14-10, nbs=64, nms=False, o

train: Scanning C:\Users\User\Documents\ABIYA NITIP\TelU-Tubes-BisnisDigital-CourtPlay-AIDevelopment\ObjectDetection\datasets\object-detection\train\labels.cache... 3686 images, 2 backgrounds, 0 corrupt: 100%|██████████| 3686/3686 [00:00<?, ?it/s]

AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 3060) 12.00G total, 1.28G reserved, 0.54G allocated, 10.17G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


     2590230       6.442         0.562         20.51          1804        (1, 3, 640, 640)                    list
     2590230       12.88         0.703         21.01          1635        (2, 3, 640, 640)                    list
     2590230       25.77         0.958            22          1391        (4, 3, 640, 640)                    list
     2590230       51.53         1.617            29          1481        (8, 3, 640, 640)                    list
     2590230       103.1         2.787         53.77          1481       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 37 for CUDA:0 7.76G/12.00G (65%) 
train: Fast image access  (ping: 0.00.0 ms, read: 1186.0295.1 MB/s, size: 185.2 KB)


train: Scanning C:\Users\User\Documents\ABIYA NITIP\TelU-Tubes-BisnisDigital-CourtPlay-AIDevelopment\ObjectDetection\datasets\object-detection\train\labels.cache... 3686 images, 2 backgrounds, 0 corrupt: 100%|██████████| 3686/3686 [00:00<?, ?it/s]


...

        2/2      5.03G      1.076     0.8113     0.9269        117        640: 100%|██████████| 100/100 [00:30<00:00,  3.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.66it/s]

                   all        527       1962      0.767      0.665      0.665      0.397



2 epochs completed in 0.021 hours.
Optimizer stripped from results\object-detection\n\10-05_14-10\weights\last.pt, 5.4MB
Optimizer stripped from results\object-detection\n\10-05_14-10\weights\best.pt, 5.4MB

Validating results\object-detection\n\10-05_14-10\weights\best.pt...
Ultralytics 8.3.179  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.91it/s]


                   all        527       1962      0.763      0.664      0.664      0.397
                  ball        408        414      0.615      0.338      0.337     0.0813
                player        526       1548      0.911      0.989       0.99      0.712
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to results\object-detection\n\10-05_14-10
MLflow: mlflow run still alive, remember to close it using mlflow.end_run()
MLflow: results logged to https://dagshub.com/abiyamf/courtPlay.mlflow
MLflow: disable with 'yolo settings mlflow=False'
🏃 View run 10-05_14-10 at: https://dagshub.com/abiyamf/courtPlay.mlflow/#/experiments/6/runs/c4250b9759d34a9c942b0aefb2dc0991
🧪 View experiment at: https://dagshub.com/abiyamf/courtPlay.mlflow/#/experiments/6


In [ ]:
model = YOLO(f"{yoloVersion}{modelSize}.pt")

mlflow.set_experiment(f"courtPlay-{modelSize}")

start_time = time.time()

with mlflow.start_run(run_name=f"{time_now}"):
    mlflow.log_params({
        "project type": projectType,
        "dataset version": datasetVersion,
        "yolo version": yoloVersion,
        "model size": modelSize,
        "gpu": gpu_name,
        "epochs": epochs,
        "batch": batch,
        "imgsz": imgsz,
    })

    results = model.train(
        data=data_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        name=f"{time_now}",
        project=project_path,
        exist_ok=True,
        mosaic=0.4,
        scale=0.25,
        translate=0.05,
        hsv_s=0.4,
        hsv_v=0.2,
        mixup=0.1,
        flipud=0.0,
        cutmix=0.0,
        shear=0.0,
        perspective=0.0
    )

    mlflow.log_params({
        "training time": format_duration(time.time() - start_time),
    })

    mlflow.log_artifact(save_class_metrics(results), artifact_path="class_metrics")
    
mlflow.end_run()

New https://pypi.org/project/ultralytics/8.3.204 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.179  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/object-detection-tile/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.2, imgsz=720, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo12m.pt, momentum=0.937, mosaic=0.4, multi_scale=False, name=10-05_09-45, nbs=64, nms=Fa

train: Scanning C:\Users\User\Documents\ABIYA NITIP\TelU-Tubes-BisnisDigital-CourtPlay-AIDevelopment\ObjectDetection\datasets\object-detection-tile\train\labels... 7372 images, 1532 backgrounds, 0 corrupt: 100%|██████████| 7372/7372 [00:10<00:00, 687.20it/s]

train: C:\Users\User\Documents\ABIYA NITIP\TelU-Tubes-BisnisDigital-CourtPlay-AIDevelopment\ObjectDetection\datasets\object-detection-tile\train\images\video-1-1-minute-_mp4-0166_jpg.rf.f68a35c0334b1a224963d06c7f453ecf.jpg: 1 duplicate labels removed


train: New cache created: C:\Users\User\Documents\ABIYA NITIP\TelU-Tubes-BisnisDigital-CourtPlay-AIDevelopment\ObjectDetection\datasets\object-detection-tile\train\labels.cache
AutoBatch: Computing optimal batch size for imgsz=736 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 3060) 12.00G total, 0.21G reserved, 0.19G allocated, 11.60G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    20139030       89.59         2.127         51.03          1131        (1, 3, 736, 736)                    list
    20139030       179.2         3.316         58.01          1180        (2, 3, 736, 736)                    list
    20139030       358.4         5.729         104.2          1207        (4, 3, 736, 736)                    list
    20139030       716.7        10.578         200.7          1326        (8, 3, 736, 736)                    list
    20139030        1433        20.055          7143     

train: Scanning C:\Users\User\Documents\ABIYA NITIP\TelU-Tubes-BisnisDigital-CourtPlay-AIDevelopment\ObjectDetection\datasets\object-detection-tile\train\labels.cache... 7372 images, 1532 backgrounds, 0 corrupt: 100%|██████████| 7372/7372 [00:00<?, ?it/s]

...

      25/25      6.01G     0.9089     0.4253      1.044          1        736: 100%|██████████| 1475/1475 [07:09<00:00,  3.44it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 211/211 [00:26<00:00,  7.93it/s]

                   all       2108       2384      0.905      0.835      0.857      0.499



25 epochs completed in 3.315 hours.
Optimizer stripped from results\object-detection\m\10-05_09-45\weights\last.pt, 40.7MB
Optimizer stripped from results\object-detection\m\10-05_09-45\weights\best.pt, 40.7MB

Validating results\object-detection\m\10-05_09-45\weights\best.pt...
Ultralytics 8.3.179  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLOv12m summary (fused): 169 layers, 20,106,454 parameters, 0 gradients, 67.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 211/211 [00:24<00:00,  8.45it/s]


                   all       2108       2384      0.905      0.835      0.857      0.498
                  ball        432        437      0.839      0.741      0.757      0.279
                player       1602       1947      0.971      0.929      0.958      0.718
Speed: 0.2ms preprocess, 8.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to results\object-detection\m\10-05_09-45
MLflow: mlflow run still alive, remember to close it using mlflow.end_run()
MLflow: results logged to https://dagshub.com/abiyamf/courtPlay.mlflow
MLflow: disable with 'yolo settings mlflow=False'
🏃 View run 10-05_09-45 at: https://dagshub.com/abiyamf/courtPlay.mlflow/#/experiments/2/runs/7fc96537f4194a5d82bc0eae8538072f
🧪 View experiment at: https://dagshub.com/abiyamf/courtPlay.mlflow/#/experiments/2
